# 1. Google Cloud: API Gateway の使い方

[Cloud Run](https://cloud.google.com/run?hl=ja) をバックエンドに、API Gateway をセットアップしてみましょう。

## 1.1. 変数・初期値の設定

本セクションで利用する環境変数・初期値を設定します。

In [ ]:
project_number="$(gcloud projects describe ${PROJECT_ID} --format='value(projectNumber)')"
api_name="my-apis"
gateway_name="prod-gateway"
service_account="${project_number}-compute@developer.gserviceaccount.com"
echo "Service account: ${service_account}"

応答例）
```text
Service Account: 12345678901-compute@developer.gserviceaccount.com
```

In [ ]:
gcloud config set compute/region asia-northeast1
gcloud config set compute/zone asia-northeast1-a
gcloud config set run/region asia-northeast1
gcloud config set run/platform managed

応答例）
```text
Updated property [compute/region].
Updated property [compute/zone].
Updated property [run/region].
Updated property [run/platform].
```

## 1.2. API の有効化

利用するサービス群を有効化します。

In [ ]:
gcloud services enable apigateway.googleapis.com
gcloud services enable servicemanagement.googleapis.com
gcloud services enable servicecontrol.googleapis.com

## 1.3. API Gateway の作成

API Gateway のバックエンドとして Cloud Run を利用します。  
コンテナに渡された環境変数や [クラウドのメタデータサーバ](https://cloud.google.com/compute/docs/storing-retrieving-metadata?hl=ja) にアクセスした結果を返す [REST API](https://github.com/pottava/http-return-everything) を Cloud Run にデプロイします。

In [ ]:
gcloud run deploy backend-apis --image gcr.io/pottava/re:v2.0 \
    --set-env-vars ENABLE_GCP=1,ENABLE_AWS=0

応答例）
```text
..
Done.                                                                          
Service [backend-apis] revision [backend-apis-00001-xxx] has been deployed and is serving 100 percent of traffic.
Service URL: https://backend-apis-xxxxxx-an.a.run.app
```

デプロイされた [Cloud Run サービスのエンドポイントをバックエンドとした API の定義](resources/v1.yaml) を作成し、API Gateway の設定を v1 として作成します。

In [ ]:
api_endpoint="$(gcloud run services describe backend-apis --format 'value(status.url)')"
find resources -type f -name "*.yaml" | xargs sed -ie "s|<backend-apis>|${api_endpoint}|g"
gcloud beta api-gateway api-configs create v1 \
    --api="${api_name}" --openapi-spec=resources/v1.yaml \
    --backend-auth-service-account="${service_account}"

[次へ: 05-teardown-resources.ipynb](05-teardown-resources.ipynb)